In [1]:
#!pip install tensorflow[and-cuda]


In [2]:
import os
import cv2
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import LSTM, Dense, GlobalAveragePooling2D, TimeDistributed, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image


In [3]:
def extract_frames(video_path, max_frames=10, target_size=(224,224)):
    frames = []
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = max(1, total_frames // max_frames)
    
    for i in range(max_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_interval)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, target_size)
        frame = frame / 255.0
        frames.append(frame)
    cap.release()
    return np.array(frames)


In [4]:
DATASET_DIR = r"C:\Users\Elvisha\OneDrive\Desktop\PBL\Dataset"

for root, dirs, files in os.walk(DATASET_DIR):
    print(root, "->", len(files), "files")


C:\Users\Elvisha\OneDrive\Desktop\PBL\Dataset -> 0 files
C:\Users\Elvisha\OneDrive\Desktop\PBL\Dataset\NonViolence -> 25 files
C:\Users\Elvisha\OneDrive\Desktop\PBL\Dataset\Violence -> 25 files


In [5]:
categories = ["NonViolence", "Violence"]


In [6]:
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical

X, y = [], []

def extract_frames(video_path, max_frames=10, target_size=(224,224)):
    frames = []
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = max(1, total_frames // max_frames)
    
    for i in range(max_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_interval)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, target_size)
        frame = frame / 255.0
        frames.append(frame)
    cap.release()
    return np.array(frames)

for label, category in enumerate(categories):
    folder = os.path.join(DATASET_DIR, category)
    for video in os.listdir(folder):
        video_path = os.path.join(folder, video)
        try:
            frames = extract_frames(video_path)
            if len(frames) == 10:
                X.append(frames)
                y.append(label)
        except Exception as e:
            print("Skipping:", video_path, "due to", e)

X = np.array(X)
y = to_categorical(np.array(y))
print(" Dataset loaded successfully")
print("X shape:", X.shape)
print("y shape:", y.shape)


 Dataset loaded successfully
X shape: (50, 10, 224, 224, 3)
y shape: (50, 2)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout

model = Sequential([
    Conv3D(32, kernel_size=(3,3,3), activation='relu', input_shape=(10,224,224,3)),
    MaxPooling3D(pool_size=(1,2,2)),
    Conv3D(64, kernel_size=(3,3,3), activation='relu'),
    MaxPooling3D(pool_size=(2,2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])


In [9]:
from tensorflow.keras.layers import TimeDistributed, LSTM

# Use a pre-trained CNN (like MobileNetV2) inside TimeDistributed for feature extraction


In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=20,
    batch_size=2
)


Epoch 1/20
18/18 [==============================] - 104s 6s/step - loss: 4.7073 - accuracy: 0.5278 - val_loss: 0.6761 - val_accuracy: 0.2500
Epoch 2/20
18/18 [==============================] - 96s 5s/step - loss: 0.6371 - accuracy: 0.6111 - val_loss: 0.6663 - val_accuracy: 0.2500
Epoch 3/20
18/18 [==============================] - 100s 6s/step - loss: 0.6176 - accuracy: 0.7778 - val_loss: 0.6189 - val_accuracy: 1.0000
Epoch 4/20
18/18 [==============================] - 113s 6s/step - loss: 0.5604 - accuracy: 0.7500 - val_loss: 0.5217 - val_accuracy: 1.0000
Epoch 5/20
18/18 [==============================] - 111s 6s/step - loss: 0.5191 - accuracy: 0.7222 - val_loss: 0.5289 - val_accuracy: 1.0000
Epoch 6/20
18/18 [==============================] - 112s 6s/step - loss: 0.4685 - accuracy: 0.7500 - val_loss: 0.4721 - val_accuracy: 1.0000
Epoch 7/20
18/18 [==============================] - 111s 6s/step - loss: 0.6957 - accuracy: 0.6667 - val_loss: 0.1392 - val_accuracy: 1.0000
Epoch 8/20
18/

In [11]:
loss, acc = model.evaluate(X_test, y_test)
print("Test accuracy:", acc)


1/1 [==============================] - 2s 2s/step - loss: 0.5244 - accuracy: 0.7000
Test accuracy: 0.699999988079071


In [16]:
import cv2

for i in range(3):  # visualize 3 test videos
    frames = X_test[i]
    pred_class = categories[np.argmax(model.predict(frames[np.newaxis]))]
    
    for frame in frames:
        frame_show = (frame * 255).astype(np.uint8)
        cv2.putText(frame_show, pred_class, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Frame', frame_show)
        if cv2.waitKey(200) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()


1/1 [==============================] - 0s 310ms/step


In [31]:
def predict_video(video_path, visualize=False):
    frames = extract_frames(video_path)
    frames_input = np.expand_dims(frames, axis=0)
    
    pred = model.predict(frames_input)[0]
    violence_prob = pred[0]
    
    if violence_prob > 0.6:
        alert = "⚠️ Fight Detected!"
    else:
        alert = "✅ No Fight Detected"
    
    print(f"Violence Probability: {violence_prob:.2f}")
    print(alert)
    
    if visualize:
        import cv2
        for frame in frames:
            frame_show = (frame * 255).astype(np.uint8)
            cv2.putText(frame_show, alert, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Video Prediction', frame_show)
            if cv2.waitKey(200) & 0xFF == ord('q'):
                break
        cv2.destroyAllWindows()
    
    return alert

# Example usage
predict_video("crowd_violence__Occupy_Cal_video_Police_brutally_beat_arrest_Berkeley_students__RussiaToday__B_f06VQOkI4.avi", visualize=True)



1/1 [==============================] - 0s 283ms/step
Violence Probability: 0.97
⚠️ Fight Detected!


In [35]:
# Save the model
model.save("fight_detection_model.h5")  # .h5 format


C:\Users\ELVISHA\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
